In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import logsumexp
import random
import pickle
import time
import sys
from pathlib import Path
import os
import torch
import gpytorch
import warnings
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import LogExpectedImprovement, UpperConfidenceBound
from botorch.optim import optimize_acqf
from botorch.models.utils import gpt_posterior_settings
from botorch.models.transforms import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
import datetime
from botorch.exceptions import OptimizationWarning, InputDataWarning, BadInitialCandidatesWarning



c:\Users\mauri\miniconda3\envs\GPBO\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

## Adding modules
sys.path.append(str(Path('./').resolve().parent.parent))
from src.utils import synthetic_datasets

## OG paper conversion

In [3]:


# Helper function to sample categorical data
def sample_categorical(prob_partition, size):
    return np.random.choice(len(prob_partition), size=size, p=prob_partition)

# Main function
def sample_struct_priors(xx, yy, fixhyp):
    print("Start sample struct priors")
    dx = xx.shape[1]
    n_partition = dx

    hyp = {}

    if all(k in fixhyp for k in ["l", "sigma", "sigma0"]):
        hyp["l"] = fixhyp["l"]
        hyp["sigma"] = fixhyp["sigma"]
        hyp["sigma0"] = fixhyp["sigma0"]
        decomp = learn_partition(xx, yy, hyp, fixhyp, n_partition)
    else:
        prob_partition = np.ones(n_partition) / n_partition
        if "z" in fixhyp:
            decomp = fixhyp["z"]
        else:
            decomp = sample_categorical(prob_partition, dx)

        num_iter = 2
        guess_params = []

        for _ in range(num_iter):
            assert max(decomp) <= n_partition

            def nll_func(params):
                return compute_nlz_wrap(xx, yy, params, n_partition, decomp)

            bounds = np.vstack([
                np.tile([0, 10], (n_partition, 1)),
                np.tile([-5, 2], (n_partition, 1)),
                np.tile([-10, 1], (n_partition, 1))
            ])

            res = minimize(
                nll_func, x0=np.zeros(bounds.shape[0]), bounds=bounds,
                method='L-BFGS-B'
            )

            best_params = res.x
            print(f"Finished optimize hyp nll={res.fun}")

            guess_params.append(best_params)

            l = np.exp(best_params[:n_partition][decomp])
            sigma = np.exp(best_params[n_partition:2*n_partition])
            sigma0 = np.exp(best_params[2*n_partition:])

            hyp["l"] = l
            hyp["sigma"] = sigma
            hyp["sigma0"] = sigma0

            decomp = learn_partition(xx, yy, hyp, fixhyp, n_partition)
            fixhyp["z"] = decomp

    return decomp, hyp

# Helper function to learn the partition
def learn_partition(xx, yy, hyp, fixhyp, n_partition):
    if "decomp" in fixhyp:
        return fixhyp["decomp"]

    N_gibbs = 10
    gibbs_iter = N_gibbs // 2
    dim_limit = 3
    maxNdata = 750

    Nidx = min(maxNdata, xx.shape[0])
    xx = xx[:Nidx]
    yy = yy[:Nidx]

    hyp_dirichlet = np.ones(n_partition) * 1
    prob_partition = hyp_dirichlet / hyp_dirichlet.sum()

    if "z" in fixhyp:
        z = fixhyp["z"]
    else:
        z = sample_categorical(prob_partition, xx.shape[1])

    z_best = None
    minnlz = float('inf')

    for i in range(N_gibbs):
        for d in range(xx.shape[1]):
            log_prob = np.full(n_partition, -np.inf)
            nlz = np.full(n_partition, float('inf'))
            
            for a in range(n_partition):
                z[d] = a

                if i >= gibbs_iter and np.sum(z == a) >= dim_limit:
                    continue

                nlz[a] = compute_nlz(xx, yy, hyp, z)
                log_prob[a] = np.log(np.sum(z == a) + hyp_dirichlet[a]) - nlz[a]

            z[d] = np.argmax(log_prob - np.log(-np.log(np.random.rand(n_partition))))

            if minnlz > nlz[z[d]]:
                z_best = z.copy()
                minnlz = nlz[z[d]]

    return z_best

# Placeholder for compute_nlz and compute_nlz_wrap
def compute_nlz(xx, yy, hyp, z):
    # Implement the calculation of the negative log likelihood here
    return 0

def compute_nlz_wrap(xx, yy, params, n_partition, decomp):
    # Wrap the negative log likelihood computation
    return 0


In [4]:
def compute_gram(xx, hyp, hyp_idx, z):
    """
    Compute the Gram matrix for an additive Gaussian process (add-GP).

    Parameters:
    - xx: numpy.ndarray
        Input data of shape (n_samples, n_features).
    - hyp: dict
        Dictionary containing hyperparameters:
        - 'l': numpy.ndarray of shape (n_partitions, n_features): length scales.
        - 'sigma': numpy.ndarray of shape (n_partitions,): signal variances.
        - 'sigma0': numpy.ndarray of shape (n_partitions,): noise variances.
    - hyp_idx: int
        Index for the hyperparameter set to use.
    - z: numpy.ndarray
        Array of shape (n_features,) defining the decomposition of input dimensions.
        Each element specifies the partition index for the corresponding feature.

    Returns:
    - K: numpy.ndarray
        Gram matrix of shape (n_samples, n_samples).
    """
    all_cat = np.unique(z)  # Unique partition indices in z
    K = 0  # Initialize Gram matrix

    for category in all_cat:
        # Get indices of features belonging to the current partition
        feature_indices = np.where(z == category)[0]
        
        # Extract relevant data and hyperparameters
        xx_partition = xx[:, feature_indices]
        print(f'This is hyp: {hyp}')
        l_partition = hyp['l'][hyp_idx, feature_indices]
        sigma_partition = hyp['sigma'][hyp_idx, category]
        sigma0_partition = hyp['sigma0'][hyp_idx, category]

        # Compute the Gram matrix for this partition and accumulate
        K += computeKmm(xx_partition, l_partition, sigma_partition, sigma0_partition)

    return K

def computeKmm(xx, l, sigma, sigma0):
    """
    Compute the covariance (Gram) matrix for a single partition of features.

    Parameters:
    - xx: numpy.ndarray
        Input data of shape (n_samples, n_features).
    - l: numpy.ndarray
        Length scales for the features.
    - sigma: float
        Signal variance.
    - sigma0: float
        Noise variance.

    Returns:
    - K: numpy.ndarray
        Gram matrix of shape (n_samples, n_samples).
    """
    # Compute squared distance matrix scaled by length scales
    scaled_xx = xx / l
    pairwise_sq_dists = np.sum(scaled_xx**2, axis=1, keepdims=True) - 2 * np.dot(scaled_xx, scaled_xx.T) + np.sum(scaled_xx**2, axis=1)

    # Compute covariance matrix using the squared exponential kernel
    K = sigma * np.exp(-0.5 * pairwise_sq_dists)

    # Add noise variance on the diagonal
    K += np.eye(K.shape[0]) * sigma0

    return K

In [5]:

def ucb_choose(xx, yy, kernel_matrix_inv, guesses, sigma0, sigma, l, xmin, xmax, alpha, beta):
    """
    Select the next evaluation point using the Upper Confidence Bound (UCB) acquisition function.

    Parameters:
    - xx: numpy.ndarray
        Observed input points of shape (n_samples, n_features).
    - yy: numpy.ndarray
        Observed output values of shape (n_samples,).
    - kernel_matrix_inv: numpy.ndarray
        Precomputed inverse Gram matrix for the Gaussian process.
    - guesses: numpy.ndarray
        Points to consider for evaluation, shape (n_guesses, n_features).
    - sigma0: float
        Noise variance.
    - sigma: float
        Signal variance.
    - l: numpy.ndarray
        Length scales of shape (n_features,).
    - xmin: numpy.ndarray
        Lower bounds of the search space, shape (n_features,).
    - xmax: numpy.ndarray
        Upper bounds of the search space, shape (n_features,).
    - alpha: float
        Exploration-exploitation balance parameter.
    - beta: float
        Scale factor for the confidence bound.

    Returns:
    - optimum: numpy.ndarray
        The selected next evaluation point, shape (n_features,).
    """

    def evaluate_ucb(x):
        """
        Compute the UCB acquisition function value for a given point x.
        
        Parameters:
        - x: numpy.ndarray
            A single input point of shape (n_features,).
        
        Returns:
        - ucb: float
            UCB acquisition function value.
        """
        x = x.reshape(1, -1)  # Ensure x is 2D

        # Compute kernel vector between x and observed data
        k = compute_kernel_vector(x, xx, l, sigma)

        # Predictive mean and variance
        mu = k.T @ kernel_matrix_inv @ yy
        sigma_sq = compute_kernel_scalar(x, l, sigma, sigma0) - k.T @ kernel_matrix_inv @ k

        # UCB value
        return -(mu + beta * np.sqrt(max(sigma_sq, 0)))  # Negative for maximization

    # Use guesses and observed data as initial candidates
    initial_points = np.vstack([guesses, xx])
    best_ucb = float('-inf')
    optimum = None

    for x0 in initial_points:
        # Constrain the optimization within bounds
        bounds = [(xmin[i], xmax[i]) for i in range(len(xmin))]

        # Optimize UCB starting from x0
        res = minimize(evaluate_ucb, x0, bounds=bounds, method='L-BFGS-B')

        if -res.fun > best_ucb:
            best_ucb = -res.fun
            optimum = res.x

    return optimum


def compute_kernel_vector(x, xx, l, sigma):
    """
    Compute the kernel vector between a single point x and a dataset xx.

    Parameters:
    - x: numpy.ndarray
        Single input point of shape (1, n_features).
    - xx: numpy.ndarray
        Dataset of shape (n_samples, n_features).
    - l: numpy.ndarray
        Length scales of shape (n_features,).
    - sigma: float
        Signal variance.

    Returns:
    - k: numpy.ndarray
        Kernel vector of shape (n_samples,).
    """
    scaled_xx = xx / l
    scaled_x = x / l
    pairwise_sq_dists = np.sum(scaled_x**2, axis=1) - 2 * np.dot(scaled_x, scaled_xx.T) + np.sum(scaled_xx**2, axis=1)
    return sigma * np.exp(-0.5 * pairwise_sq_dists)


def compute_kernel_scalar(x, l, sigma, sigma0):
    """
    Compute the kernel scalar (self-covariance) for a single point x.

    Parameters:
    - x: numpy.ndarray
        Single input point of shape (1, n_features).
    - l: numpy.ndarray
        Length scales of shape (n_features,).
    - sigma: float
        Signal variance.
    - sigma0: float
        Noise variance.

    Returns:
    - k_scalar: float
        Kernel scalar value.
    """
    return sigma + sigma0


In [6]:

def add_gpopt(objective, xmin, xmax, T, initx=None, inity=None, options=None):
    """
    Maximize the function objective using Bayesian Optimization with additive Gaussian Processes.

    Parameters:
    - objective: callable, the function to optimize.
    - xmin, xmax: array-like, the bounds of the search space.
    - T: int, the number of sequential evaluations.
    - initx, inity: array-like, initial observed inputs and outputs.
    - options: dict, additional options.

    Returns:
    - results: dict containing inferred argmax points, function values, evaluated points, and timing.
    """
    # Default options
    if options is None:
        options = {}
    options.setdefault('restart', 0)
    options.setdefault('savefilenm', None)
    options.setdefault('noiselevel', 0)
    options.setdefault('nK', 1)
    options.setdefault('nFeatures', 10000)
    options.setdefault('seed', 42)
    options.setdefault('learn_interval', 10)

    np.random.seed(options['seed'])

    if options['restart'] and options['savefilenm'] and os.path.exists(options['savefilenm']):
        with open(options['savefilenm'], 'rb') as f:
            results = pickle.load(f)
        xx = results['xx']
        yy = results['yy']
        choose_time = results['choose_time']
        extra_time = results['extra_time']
        t_start = results['t']
        z = results['z']
    else:
        t_start = 0
        if initx is None or inity is None:
            initx = np.random.uniform(xmin, xmax, (1, len(xmin)))
            inity = objective(initx)

        xx = initx
        yy = inity
        choose_time = []
        extra_time = []

    for t in range(t_start + 1, T + 1):
        # Learn structure
        if t % options['learn_interval'] == 1:
            start_time = time.time()
            z, hyp = sample_struct_priors(xx, yy, options)
            options['z'] = z
            extra_time.append(time.time() - start_time)

        # Choose next point
        start_time = time.time()
        kernel_matrix = compute_gram(xx, hyp, 1, z)
        kernel_matrix_inv = np.linalg.inv(kernel_matrix)

        x_next = np.zeros_like(xx[0])
        all_categories = np.unique(z)

        for cat in all_categories:
            coords = z == cat
            xx_sub = xx[:, coords]
            xmin_sub = xmin[coords]
            xmax_sub = xmax[coords]
            l = hyp['l'][:, coords]
            sigma = hyp['sigma'][cat]
            sigma0 = hyp['sigma0'][cat]
            alpha = 1
            beta = np.sqrt(len(xx_sub[0]) * np.log(2 * t) / 5)

            optimum = ucb_choose(xx_sub, yy, kernel_matrix_inv, sigma0, sigma, l, xmin_sub, xmax_sub, alpha, beta)
            x_next[coords] = optimum

        choose_time.append(time.time() - start_time)

        xx = np.vstack([xx, x_next])
        yy = np.vstack([yy, objective(x_next) + np.random.normal(0, options['noiselevel'], size=(1,))])

        print(f"{t}: val={yy[-1][0]}")

        if options['savefilenm'] and t % 10 == 0:
            results = {
                'xx': xx,
                'yy': yy,
                'choose_time': choose_time,
                'extra_time': extra_time,
                't': t,
                'z': z
            }
            with open(options['savefilenm'], 'wb') as f:
                pickle.dump(results, f)

    return {
        'xx': xx,
        'yy': yy,
        'choose_time': choose_time,
        'extra_time': extra_time
    }

# Supporting functions (e.g., sample_struct_priors, compute_gram, ucb_choose) should also be implemented.
# These functions involve translating the specific mathematical operations in MATLAB to Python using NumPy or SciPy.


In [7]:

# Define function to sample from additive Gaussian Process (add-GP)
def sample_addGP(dx, n_samples, xmin, xmax):
    """
    Placeholder for the sample_addGP function. This should sample a function
    from an additive Gaussian Process.

    Parameters:
    - dx: int, number of dimensions.
    - n_samples: int, number of samples to generate.
    - xmin: numpy.ndarray, lower bounds of the input space.
    - xmax: numpy.ndarray, upper bounds of the input space.

    Returns:
    - function: Callable that takes an input and evaluates the sampled add-GP.
    """
    # Placeholder implementation (replace with your actual function)
    def sampled_function(x):
        return np.sum(np.sin(x), axis=-1)

    return sampled_function


# Define main script logic
def main():
    # Set parameters
    dx = 20
    xmin = np.zeros(dx)
    xmax = np.ones(dx)

    # Sample function from add-GP
    f = sample_addGP(dx, dx, xmin, xmax)

    # Set options
    options = {
        "savefilenm": None,  # Placeholder for save file name
        "nK": 5,  # Number of maximums to sample
        "l": np.ones(dx) * 50,  # Length scale hyperparameter
        "sigma": np.ones(dx) * 5,  # Signal variance
        "sigma0": np.ones(dx) * 0.0001,  # Noise variance
    }

    # Start Bayesian Optimization with Add-GP
    add_gpopt(f, xmin, xmax, 200,options=options)



## Own implementation

In [3]:
class GP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(GP, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [133]:
# Helper function to sample categorical data
def sample_categorical(prob_partition, size):
    #e.g. sample_cateogircal([0.1, 0.1, 0.4, 0.2, 0.2], 4) = array([2, 2, 2, 3])
    return np.random.choice(len(prob_partition), size=size, p=prob_partition)

# Main function
def sample_struct_priors(xx, yy, fixhyp):
    print("Start sample struct priors")
    dx = xx.shape[1]
    n_partition = dx

    hyp = {}

    if all(k in fixhyp for k in ["l", "sigma", "sigma0"]):
        hyp["l"] = fixhyp["l"]
        hyp["sigma"] = fixhyp["sigma"]
        hyp["sigma0"] = fixhyp["sigma0"]
        decomp = learn_partition(xx, yy, hyp, fixhyp, n_partition)
    else:
        prob_partition = np.ones(n_partition) / n_partition
        decomp = fixhyp.get("z", sample_categorical(prob_partition, dx))
        
        '''
        num_iter = 2
        for _ in range(num_iter):
            assert max(decomp) <= n_partition

            def compute_mll(params):
                hyp = {
                    "l": np.exp(params[:n_partition]),
                    "sigma": np.exp(params[n_partition:2 * n_partition]),
                    "sigma0": np.exp(params[2 * n_partition:])
                }
                return compute_nlz(xx, yy, hyp, decomp)

            bounds = np.vstack([
                np.tile([0, 10], (n_partition, 1)),
                np.tile([-5, 2], (n_partition, 1)),
                np.tile([-10, 1], (n_partition, 1))
            ])

            res = minimize(
                compute_mll, x0=np.zeros(bounds.shape[0]), bounds=bounds,
                method='L-BFGS-B'
            )

            best_params = res.x
            #print(f"Finished optimize hyp nll={res.fun}")

            l = np.exp(best_params[:n_partition][decomp])
            sigma = np.exp(best_params[n_partition:2*n_partition])
            sigma0 = np.exp(best_params[2*n_partition:])

            hyp["l"] = l
            hyp["sigma"] = sigma
            hyp["sigma0"] = sigma0

            decomp = learn_partition(xx, yy, hyp, fixhyp, n_partition)
            fixhyp["z"] = decomp
        '''

        # GPytorch Model Training
        #model = SingleTaskGP(xx, yy, outcome_transform=Standardize(m=1))
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        model = GP(xx, yy, likelihood, gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims= dx)))
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

        training_iter = 50
        model.train()
        likelihood.train()

        for i in range(training_iter):
            optimizer.zero_grad()
            output = model(model.train_inputs[0])
            loss = -mll(output, model.train_targets).mean()
            loss.backward()
            optimizer.step()

        # Extract optimized hyperparameters
        hyp["l"] = model.covar_module.base_kernel.lengthscale.detach().numpy().squeeze()
        hyp["sigma"] = model.covar_module.outputscale.detach().numpy()
        hyp["sigma0"] = model.likelihood.noise.detach().numpy()

        decomp = learn_partition(xx, yy, hyp, fixhyp, n_partition)
        fixhyp["z"] = decomp
        

    return decomp, hyp

# Helper function to learn the partition
def learn_partition(xx, yy, hyp, fixhyp, n_partition):
    if "decomp" in fixhyp:
        return fixhyp["decomp"]

    N_gibbs = 10
    gibbs_iter = N_gibbs // 2
    dim_limit = 3
    maxNdata = 750

    Nidx = min(maxNdata, xx.shape[0])
    xx = xx[:Nidx]
    yy = yy[:Nidx]

    hyp_dirichlet = np.ones(n_partition)
    prob_partition = hyp_dirichlet / hyp_dirichlet.sum()

    z = fixhyp.get("z", sample_categorical(prob_partition, xx.shape[1]))

    z_best = z.copy()
    minnlz = float('inf')

    for i in range(N_gibbs):
        for d in range(xx.shape[1]):
            log_prob = np.full(n_partition, -np.inf)
            nlz = np.full(n_partition, float('inf'))

            for a in range(n_partition):
                z[d] = a

                if i >= gibbs_iter and np.sum(z == a) >= dim_limit:
                    continue

                nlz[a] = compute_nlz(xx, yy, hyp, z)
                log_prob[a] = np.log(np.sum(z == a) + hyp_dirichlet[a]) - nlz[a]

            z[d] = np.argmax(log_prob - np.log(-np.log(np.random.rand(n_partition))))

            if minnlz > nlz[z[d]]:
                z_best = z.copy()
                minnlz = nlz[z[d]]

    return z_best

def compute_nlz(xx, yy, hyp, z):
    """
    Compute the negative log likelihood (NLL) for the given decomposition.
    """
    model = SingleTaskGP(torch.tensor(xx), torch.tensor(yy), outcome_transform=Standardize(m=1))
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    model.train()
    mll_value = -mll(model(model.train_inputs[0]), model.train_targets)
    return mll_value.item()


In [129]:
def compute_gram(xx, hyp, z):
    """
    Compute the Gram matrix for an additive Gaussian process (add-GP).

    Parameters:
    - xx: numpy.ndarray
        Input data of shape (n_samples, n_features).
    - hyp: dict
        Dictionary containing hyperparameters:
        - 'l': numpy.ndarray of shape (n_partitions, n_features): length scales.
        - 'sigma': numpy.ndarray of shape (n_partitions,): signal variances.
        - 'sigma0': numpy.ndarray of shape (n_partitions,): noise variances.
    - hyp_idx: int
        Index for the hyperparameter set to use.
    - z: numpy.ndarray
        Array of shape (n_features,) defining the decomposition of input dimensions.
        Each element specifies the partition index for the corresponding feature.

    Returns:
    - K: numpy.ndarray
        Gram matrix of shape (n_samples, n_samples).
    """
    all_cat = np.unique(z)  # Unique partition indices in z
    K = 0  # Initialize Gram matrix

    for category in all_cat:
        # Get indices of features belonging to the current partition
        feature_indices = np.where(z == category)[0]
        
        # Extract relevant data and hyperparameters
        xx_partition = xx[:, feature_indices]
        l_partition = hyp['l'][feature_indices]
        #print(f'This is hyp: {hyp}')
        sigma_partition = hyp['sigma'] # TODO: Adapt so that each dimension can learn its own signal variance hyp['sigma'][category]
        sigma0_partition = hyp['sigma0'] # TODO: Adapt so that each dimension can learn its own noise variance hyp['sigma0'][category]

        # Compute the Gram matrix for this partition and accumulate
        K += computeKmm(xx_partition, l_partition, sigma_partition, sigma0_partition)

    return K

def computeKmm(xx, l, sigma, sigma0):
    """
    Compute the covariance (Gram) matrix for a single partition of features.

    Parameters:
    - xx: numpy.ndarray
        Input data of shape (n_samples, n_features).
    - l: numpy.ndarray
        Length scales for the features.
    - sigma: float
        Signal variance.
    - sigma0: float
        Noise variance.

    Returns:
    - K: numpy.ndarray
        Gram matrix of shape (n_samples, n_samples).
    """
    # Compute squared distance matrix scaled by length scales
    scaled_xx = (xx / l).numpy()
    pairwise_sq_dists = np.sum(scaled_xx**2, axis=1, keepdims=True) - 2 * np.dot(scaled_xx, scaled_xx.T) + np.sum(scaled_xx**2, axis=1)
    # Compute covariance matrix using the squared exponential kernel
    K = sigma * np.exp(-0.5 * pairwise_sq_dists)

    # Add noise variance on the diagonal
    K += np.eye(K.shape[0]) * sigma0

    return K

In [46]:
def ucb_choose(xx, yy, kernel_matrix_inv, sigma0, sigma, l, xmin, xmax, kappa):
    """
    Select the next evaluation point using the Upper Confidence Bound (UCB) acquisition function.

    Parameters:
    - xx: numpy.ndarray
        Observed input points of shape (n_samples, n_features).
    - yy: numpy.ndarray
        Observed output values of shape (n_samples,).
    - kernel_matrix_inv: numpy.ndarray
        Precomputed inverse Gram matrix for the Gaussian process.
    - sigma0: float
        Noise variance.
    - sigma: float
        Signal variance.
    - l: numpy.ndarray
        Length scales of shape (n_features,).
    - xmin: numpy.ndarray
        Lower bounds of the search space, shape (n_features,).
    - xmax: numpy.ndarray
        Upper bounds of the search space, shape (n_features,).
    - kappa: float
        Scale factor for the confidence bound.

    Returns:
    - optimum: numpy.ndarray
        The selected next evaluation point, shape (n_features,).
    """

    def evaluate_ucb(x):
        """
        Compute the UCB acquisition function value for a given point x.
        
        Parameters:
        - x: numpy.ndarray
            A single input point of shape (n_features,).
        
        Returns:
        - ucb: float
            UCB acquisition function value.
        """
        x = x.reshape(1, -1)  # Ensure x is 2D

        # Compute kernel vector between x and observed data
        k = compute_kernel_vector(x, xx, l, sigma).flatten()
        yy_detached = yy.detach().numpy().flatten()

        # Predictive mean and variance
        mu = k.T @ kernel_matrix_inv @ yy_detached
        sigma_sq = compute_kernel_scalar(x, l, sigma, sigma0) - k.T @ kernel_matrix_inv @ k

        # UCB value
        return -(mu + kappa * np.sqrt(max(sigma_sq, 0)))  # Negative for maximization

    # Use guesses and observed data as initial candidates
    best_ucb = float('-inf')
    optimum = None

    for x0 in xx:
        # Constrain the optimization within bounds
        bounds = [(xmin[i], xmax[i]) for i in range(len(xmin))]
        # Optimize UCB starting from x0
        res = minimize(evaluate_ucb, x0, bounds=bounds, method='L-BFGS-B', tol=1e-6)

        #print(f'This is res: {res}')

        if -res.fun > best_ucb:
            best_ucb = -res.fun
            optimum = res.x

    return optimum

def compute_kernel_vector(x, xx, l, sigma):
    """
    Compute the kernel vector between a single point x and a dataset xx.

    Parameters:
    - x: numpy.ndarray
        Single input point of shape (1, n_features).
    - xx: numpy.ndarray
        Dataset of shape (n_samples, n_features).
    - l: numpy.ndarray
        Length scales of shape (n_features,).
    - sigma: float
        Signal variance.

    Returns:
    - k: numpy.ndarray
        Kernel vector of shape (n_samples,).
    """
    scaled_xx = (xx / l).numpy()
    scaled_x = x / l
    pairwise_sq_dists = np.sum(scaled_x**2, axis=1) - 2 * np.dot(scaled_x, scaled_xx.T) + np.sum(scaled_xx**2, axis=1)
    return sigma * np.exp(-0.5 * pairwise_sq_dists)

def compute_kernel_scalar(x, l, sigma, sigma0):
    """
    Compute the kernel scalar (self-covariance) for a single point x.

    Parameters:
    - x: numpy.ndarray
        Single input point of shape (1, n_features).
    - l: numpy.ndarray
        Length scales of shape (n_features,).
    - sigma: float
        Signal variance.
    - sigma0: float
        Noise variance.

    Returns:
    - k_scalar: float
        Kernel scalar value.
    """
    # Compute the squared norm of the scaled input
    scaled_norm = np.sum((x / l) ** 2)

    # Compute kernel scalar
    k_scalar = sigma ** 2 * np.exp(-0.5 * scaled_norm) + sigma0 ** 2

    return k_scalar


In [130]:
def add_gp2(syn, options=None):
    """
    Maximize the function objective using Bayesian Optimization with additive Gaussian Processes.

    Parameters:
    - objective: callable, the function to optimize.
    - xmin, xmax: array-like, the bounds of the search space.
    - T: int, the number of sequential evaluations.
    - initx, inity: array-like, initial observed inputs and outputs.
    - options: dict, additional options.

    Returns:
    - results: dict containing inferred argmax points, function values, evaluated points, and timing.
    """

    # Default options
    if options is None:
        options = {}
    options.setdefault('savefilenm', None)
    options.setdefault('nK', 1)
    options.setdefault('nFeatures', 10000)
    options.setdefault('seed', 42)
    options.setdefault('learn_interval', 10)
    options.setdefault('n_iters', 200)
    options.setdefault('learning rate', 1e-3)
    options.setdefault('training iterations', 50)
    options.setdefault('acq_f', 'UCB')

    np.random.seed(options['seed'])

    #TODO: Add start of BO using model savefilenm
    t_start = 1
    xx, yy = syn.simulate(49)
    x_next, y_next = syn.simulate(1)
    yy = yy.unsqueeze(-1)
    y_next = y_next.unsqueeze(-1)
    T = options['n_iters']
    kappa = options['kappa']
    train_time = []
    choose_time = []
    extra_time = []
    predictions = []
    losses = np.zeros((T, 1))
    exploration = np.zeros(T)
    exploitation = np.zeros(T)
    mse = np.zeros((T, 1))


    for t in range(t_start, T):

        
        # Innerloop for kernel learning
        if t % options['learn_interval'] == 1:
            start_time = time.time()
            z, hyp = sample_struct_priors(xx, yy, options)
            options['z'] = z
            extra_time.append(time.time() - start_time)
            print(f'Iter {t} - Decomposition of inputs: {z}')

        #Training
        start_time = time.time()
        kernel_matrix = compute_gram(xx, hyp, z)
        kernel_matrix_inv = np.linalg.inv(kernel_matrix)

        all_cat = np.unique(z)
        x_next = np.zeros_like(xx[0])
        for cat in all_cat:
            coords = z == cat
            xx_sub = xx[:, coords]
            xmin_sub = syn.lower_bounds[coords]
            xmax_sub = syn.upper_bounds[coords]
            l = hyp['l'][coords]
            sigma = hyp['sigma']#[cat]
            sigma0 = hyp['sigma0']#[cat]
            beta = np.sqrt(len(xx_sub[0]) * np.log(2 * t) / 5)
            optimum = ucb_choose(xx_sub, yy, kernel_matrix_inv, sigma0, sigma, l, xmin_sub, xmax_sub, beta)
            x_next[coords] = optimum

        train_time.append(time.time() -start_time)
        x_next = torch.from_numpy(x_next)
        y_next = syn.f.forward(x_next).unsqueeze(-1)
        xx = torch.cat([xx, x_next.unsqueeze(0)], dim=0)
        yy = torch.cat([yy, y_next.unsqueeze(0)], dim=0)

        #Calculate metrics
        predictions.append(y_next)
        #with gpt_posterior_settings():
        #    posterior= gp.posterior(xx)
        #    y_preds = posterior.mean
        #y_trues = syn.f.forward(xx).unsqueeze(-1)
        #distances = (y_trues - y_preds)**2
        #mse[t, 0] = distances.mean()
        exploitation[t] = syn.f.forward(x_next) / syn.f.optimal_value
        best_x = xx[torch.argmin(yy)]
        exploration[t] = syn.f.forward(best_x) / syn.f.optimal_value
        
        if t % 5 == 0:
            print(f'Iteration {t}: predicted {y_next.item()} at {x_next}')

    results = {
        'xx': xx,
        'yy': yy,
        'train_time': train_time,
        'choose_time': choose_time,
        'extra_time': extra_time,
        #'loss': loss,
        'predictions': predictions,
        'exploration': exploration,
        'exploitation': exploitation,
        'mse': mse,

    }

    #save to logs
    if options['savefilenm']:
        
        with open(options['savefilenm'], 'wb') as f:
            pickle.dump(results, f)

    return results


In [136]:
def main():

    synthetic = synthetic_datasets.SyntheticTestFun('ackley', 10, 0.0, negate=True)

    options = {
        "savefilenm": '../../output/experiments/structural_kernal_'+datetime.date.today().strftime("%y%m%d")+'.pkl', # Placeholder for save file name
        "n_iters": 200,
        "l": 50,  # Length scale hyperparameter,
        "kappa": 5,
        "acq_f": "UCB",
    }

    #Start BO with Add-GP
    return add_gp2(synthetic, options=options)

In [137]:
warnings.filterwarnings("ignore", category=InputDataWarning)
warnings.filterwarnings("ignore", category=OptimizationWarning)
warnings.filterwarnings("ignore", category=BadInitialCandidatesWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
results = main()